<a href="https://colab.research.google.com/github/noallynoclan/colab/blob/master/stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [251]:
#! pip install yfinance -q
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
import yfinance as yf
from fbprophet import Prophet
from functools import lru_cache
from IPython.display import clear_output
from matplotlib.colors import TABLEAU_COLORS
from matplotlib.ticker import FuncFormatter
from scipy import stats

plt.style.use('fivethirtyeight')

def get_outliers(s):
    prev = np.log(s / s.shift(1)).fillna(0)
    next = np.log(s.shift(-1) / s).fillna(0)
    return np.min(np.abs([prev, next]), axis=0) > 1

In [231]:
@lru_cache(maxsize=1024)
def get_ticker(ticker):
    data = yf.Ticker(ticker).history(period="max")
    data = data[~get_outliers(data['Close'])].copy()
    df = pd.DataFrame({'ds': pd.date_range(data.index.min(), pd.Timestamp.today())})
    df['close'] = df['ds'].map(data['Close']).ffill()
    df = df[df['ds'].dt.weekday == 0].reset_index(drop=True)
    return df


@lru_cache(maxsize=1024)
def fit_one(ticker, n, periods):
    df = get_ticker(ticker)
    df['y'] = np.log(df['close'])
    m = Prophet(n_changepoints=n, yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
    m.fit(df)
    clear_output()
    ft = m.make_future_dataframe(periods=periods)
    fc = m.predict(ft[ft['ds'].dt.weekday == 0])
    return np.exp(fc.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']]).reset_index()

In [267]:
def fit(ticker, periods=3650):
    df = get_ticker(ticker)
    N = (df['ds'].dt.year.max() - df['ds'].dt.year.min()) // 4
    fc = pd.concat([fit_one(ticker, n, periods) for n in range(N)]).groupby('ds').agg(stats.hmean)
    fc['close'] = fc.index.map(df.set_index('ds')['close'])
    return fc

def plot(tickers):
    data = pd.concat([fit(t) for t in tickers], keys=tickers)
    fig = go.Figure()
    fig.update_layout(width=1200, height=800)
    for t, c in zip(tickers, TABLEAU_COLORS.values()):
        df = data.loc[t]
        pars = {'legendgroup': t, 'line_color': c}
        fig.add_trace(go.Scatter(
            x=[*df.index, *df.index[::-1]],
            y=[*df['yhat_upper'], *df['yhat_lower'][::-1]],
            mode='none',
            fill='toself',
            opacity=0.2,
            fillcolor=c,
            name=t,
            **pars, 
        )),
        fig.add_trace(go.Scatter(x=df.index, y=df['yhat'], name=f'{t} - pred', showlegend=False, **pars))
        fig.add_trace(go.Scatter(x=df.index, y=df['close'], name=f'{t} - actual', showlegend=False, **pars))
    fig.update_layout(yaxis_type="log")
    fig.show()
    return data

df = plot([
    'USDRUB=X', 
    '^GSPC',
    'AAPL', 
    'AMD', 
    'AMZN', 
    # 'GOOGL', 
    # 'MSFT', 
    # 'NFLX', 
    'NVDA', 
    'TSLA'
])